# Recomendation system
## Content based filtering

In [15]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from evaluation import calculate_mse_and_rmse

In [16]:
ratings = pd.read_csv('data/ratings.csv', low_memory=False)
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [17]:
movie_metadata = pd.read_csv("data/movies_metadata.csv", low_memory=False)
movie_metadata = movie_metadata[['id', 'title']]
movie_metadata = movie_metadata.drop([19730, 29503, 35587])
movie_metadata = movie_metadata[pd.to_numeric(movie_metadata['id'], errors='coerce').notnull()]
movie_metadata = movie_metadata.rename(columns={'id': 'movieId'})
movie_metadata['movieId'] = movie_metadata['movieId'].astype(int)
movie_metadata.head()

,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [18]:
movie_data = ratings.merge(movie_metadata, how='left', on='movieId')
movie_data.head()

,userId,movieId,rating,title
0,1,110,1.0,Three Colors: Red
1,1,147,4.5,The 400 Blows
2,1,858,5.0,Sleepless in Seattle
3,1,1221,5.0,NaN
4,1,1246,5.0,Rocky Balboa


In [19]:
df_movies_cnt = pd.DataFrame(ratings.groupby('movieId').size(), columns=['count'])
popular_movies = list(set(df_movies_cnt.query('count <= 10').index))
movies_filter = ratings.movieId.isin(popular_movies).values

df_users_cnt = pd.DataFrame(ratings.groupby('userId').size(), columns=['count'])
active_users = list(set(df_users_cnt.query('count <= 5').index))
users_filter = ratings.userId.isin(active_users).values

df_ratings_filtered = ratings[movies_filter & users_filter]
df_ratings_filtered.head()

,userId,movieId,rating
139044,1407,167840,5.0
461364,4783,129425,3.5
707620,7191,82395,5.0
1219398,12524,64288,3.0
1352121,13906,174403,4.5


In [28]:
reader = Reader()
ratings_by_users = Dataset.load_from_df(df_ratings_filtered[['userId', 'movieId', 'rating']], reader)
train_df, test_df = train_test_split(ratings_by_users, test_size=.2)

<class 'surprise.trainset.Trainset'>


In [22]:
svd_model = SVD()
svd_model_trained = svd_model.fit(train_df)

In [23]:
def get_recommendations(user_id):
    pred_series = []
    users_ratings = ratings[ratings['userId'] == user_id]
    for movie_id, name in zip(movie_metadata.index, movie_metadata['title']):
        rating_real = ratings.query(f'movieId == {movie_id}')['rating'].values[0] if movie_id in users_ratings[
            'movieId'].values else 0
        rating_pred = svd_model_trained.predict(user_id, movie_id, rating_real, verbose=False)
        pred_series.append([movie_id, name, rating_pred.est, rating_real])

    df_recommendations = pd.DataFrame(pred_series, columns=['movieId', 'title', 'predicted_rating', 'actual_rating'])
    return df_recommendations.sort_values(by='predicted_rating', ascending=False).head(10)

In [24]:
get_recommendations(222)

,movieId,title,predicted_rating,actual_rating
3226,3226,"Charlie, the Lonesome Cougar",3.926413,0.0
3382,3382,Network,3.847966,0.0
0,0,Toy Story,3.828704,0.0
30310,30314,The World Made Straight,3.828704,0.0
30302,30306,Festival,3.828704,0.0
30303,30307,The Prospects,3.828704,0.0
30304,30308,The Colour Out of Space,3.828704,0.0
30305,30309,Foxtrot,3.828704,0.0
30306,30310,Ghost Graduation,3.828704,0.0
30307,30311,Duck Amuck,3.828704,0.0


In [25]:
test_df = [{'userId': obj[0], 'movieId': obj[1], 'rating': obj[2]} for obj in test_df]
test_df = pd.DataFrame(test_df)

In [26]:
data = []
ratings_test = pd.merge(pd.DataFrame(test_df), movie_metadata[['movieId']], left_on='movieId', right_on='movieId', how='inner')

for index, row in ratings_test.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    rating = row['rating']
    predicted = svd_model_trained.predict(user_id, movie_id, rating, verbose=False).est
    data.append((rating, predicted))

In [27]:
mse, rmse = calculate_mse_and_rmse(data)
print("MSE:", mse)
print("RMSE:", rmse)

MSE: 1.371935013717421
RMSE: 1.1712962962962963
